# Notebook to analyze various statistics of the labeled events
# Furthermore, the events were visually inspected in the labeling notebooks to ensure their quality

## Import and Initialize Everything

In [ ]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import h5py
import pandas as pd
import os
import sys
from pathlib import Path
from datetime import datetime
from datetime import timedelta
import math
import pdb
import scipy
# Add project path to path for import
project_path = os.path.abspath("..")
if project_path not in sys.path:
    sys.path.append(project_path)

# Add module path to path for import
module_path = os.path.abspath("../data_utility/data_utility.py")
if module_path not in sys.path:
    sys.path.append(module_path)
    
from data_utility import CREAM_Day # class to work with a day of the CREAM Dataset
import seaborn as sns
%load_ext autoreload
# Reload all modules every time before executing the Python code typed.
%autoreload 2 
# Import some graphical modules
from IPython.display import display, clear_output
from ipywidgets import Button, Layout, ButtonStyle, HBox, VBox, widgets, Output
from IPython.display import SVG, display, clear_output
%matplotlib widget
import subprocess
import glob

In [ ]:
machine = "X8"
PATH_TO_DATA = os.path.abspath(os.path.join("..", "..","rbgstorage", "nilm", "i13-dataset", "CREAM", machine))
ALL_DAYS = glob.glob(os.path.join(PATH_TO_DATA, "*"))
ALL_DAYS = [os.path.basename(d) for d in ALL_DAYS if "2018" in os.path.basename(d)  or "2019" in os.path.basename(d)  ]
ALL_DAYS.sort()

## Load the Event Files

In [ ]:
#necessary for the plotting
# Load the events 
day_path = os.path.join(PATH_TO_DATA, ALL_DAYS[0]) #arbitrary day to initialize the object
current_CREAM_day = CREAM_Day(cream_day_location=day_path,use_buffer=True, buffer_size_files=2) 

if machine == "X9":
    all_component_events_fine = current_CREAM_day.load_component_events(os.path.join(PATH_TO_DATA, "component_events_fine.csv"), filter_day=False)
    all_component_events_coarse = current_CREAM_day.load_component_events(os.path.join(PATH_TO_DATA, "component_events_coarse.csv"), filter_day=False)

else:
    all_component_events = current_CREAM_day.load_component_events(os.path.join(PATH_TO_DATA, "component_events.csv"), filter_day=False)

# Load the product and the maintenance events (the raw ones, per minute events) and filter for the day
all_maintenance_events = current_CREAM_day.load_machine_events(os.path.join(PATH_TO_DATA, "maintenance_events.csv"), raw_file=False, filter_day=False)
all_product_events = current_CREAM_day.load_machine_events(os.path.join(PATH_TO_DATA, "product_events.csv"), raw_file=False, filter_day=False)

# Analyze the Event Types

#### Display the different event types and labeled components, together with their cardinality

In [ ]:
for events_df in [all_maintenance_events, all_product_events]:  
    print(events_df.Event_Type.value_counts())
    print("-------------------------------------")
all_component_events_fine.Component.value_counts()
print("-------------------------------------")
all_component_events_coarse.Component.value_counts() 

### Functions necessary to do so

In [ ]:
def plot_event_durations(events_df:pd.DataFrame, event_type_column: str = "Event_Type"):
    """
    Function to plot the event duration, for every event type.
    Parameters
    ----------
    events_df (pd.DataFrame): maintenance events or product events dataframe
    event_type_column (str): Name of the column containing the event type. 
    
    Returns
    -------
    
        
    """
                               
    for e_type in np.unique(events_df[event_type_column]):
        x = events_df[events_df[event_type_column] == e_type].Event_Duration_Seconds
        
        n_samples = len(events_df[events_df[event_type_column] == e_type])
        mean = np.mean(x)
        stdev = np.std(x)
                
            
        sns.distplot(x, label=e_type)
        plt.legend()
        plt.show()

def print_event_duration_statistics(events_df:pd.DataFrame, event_type_column: str = "Event_Type"):
    """
    Function to print the event duration, for every event type.
    Parameters
    ----------
    events_df (pd.DataFrame): maintenance events or product events dataframe
    event_type_column (str): Name of the column containing the event type. 
    
    Returns
    -------
    
        
    """
    
    data = { "event type" : [],
        "samples": [],
           "mean" : [],
           "standard deviation" : []}
    
                               
    for e_type in np.unique(events_df[event_type_column]):
        x = events_df[events_df[event_type_column] == e_type].Event_Duration_Seconds
        
        n_samples = len(events_df[events_df[event_type_column] == e_type])
        mean = np.mean(x)
        stdev = np.std(x)
        
        data["samples"].append(n_samples)
        data["mean"].append(mean)
        data["standard deviation"].append(stdev)
        data["event type"].append(e_type)
    data = pd.DataFrame(data)
    data = data.sort_values(["samples"], ascending=False)
    print(data.round(2).to_latex(index=False))
       

## Product Events

### Event Durations

In [ ]:
print_event_duration_statistics(all_product_events, "Event_Type")
print_event_duration_statistics(all_maintenance_events, "Event_Type")

In [ ]:
plot_event_durations(all_product_events, "Event_Type")

## Maintenance Events

### Event Durations

In [ ]:
plot_event_durations(all_maintenance_events, "Event_Type", NOMINAL_TIMES_PER_EVENT)

# Event Distribution per Hour

In [ ]:
def create_time_bin(hours : float, minutes : float) -> str:
    """
    Creates a hour:minutes timestamp, ceiled to full 30 minutes.
    All minutes below 15, become 0.
    All between 15 and 45 minutes, become 30 minutes.
    All minutes between 45 and 60 become 0 and belong to the next hour.
    """
    if minutes < 15:
        minutes = "00"
    elif minutes >= 15 and minutes < 45:
        minutes = "30"
    elif minutes >= 45:
        minutes = "00"
        hours += 1
    
    if hours < 10:
        hours = "0" + str(hours)
    else:
        hours = str(hours)
    
    return hours + ":" + minutes

In [ ]:
# create a new column with: hour:30, hour:0 in it for the x-axis as the labels
all_product_events["Time_Bin"] = all_product_events.Start_Timestamp.apply(lambda x: create_time_bin(x.hour, x.minute))

times, counts = np.unique(all_product_events["Time_Bin"], return_counts=True)

plt.figure(figsize=(16,4))
plt.title("Product Events")
plt.xlabel("Time")
plt.ylabel("Number of Events")
sns.barplot(x=times, y=counts, color="b")
plt.show()

# create a new column with: hour:30, hour:0 in it for the x-axis as the labels
all_maintenance_events["Time_Bin"] = all_maintenance_events.Start_Timestamp.apply(lambda x: create_time_bin(x.hour, x.minute))

times, counts = np.unique(all_maintenance_events["Time_Bin"], return_counts=True)

plt.figure(figsize=(16,4))
plt.title("Maintenance Events")
plt.xlabel("Time")
plt.ylabel("Number of Events")
sns.barplot(x=times, y=counts,color="b")
plt.show()

# create a new column with: hour:30, hour:0 in it for the x-axis as the labels
all_maintenance_events["Time_Bin"] = all_maintenance_events.Start_Timestamp.apply(lambda x: create_time_bin(x.hour, x.minute))
all_product_events["Time_Bin"] = all_product_events.Start_Timestamp.apply(lambda x: create_time_bin(x.hour, x.minute))

times, counts = np.unique(np.concatenate([all_product_events["Time_Bin"],all_maintenance_events["Time_Bin"]]), return_counts=True)

plt.figure(figsize=(16,4))
plt.title("Product and Maintenance Events")
plt.xlabel("Time")
plt.ylabel("Number of Events")
sns.barplot(x=times, y=counts,color="b")
plt.show()


fontdict_text = {"size" : 18}
# create a new column with: hour:30, hour:0 in it for the x-axis as the labels
all_component_events["Time_Bin"] = all_component_events.Timestamp.apply(lambda x: create_time_bin(x.hour, x.minute))

times, counts = np.unique(all_component_events["Time_Bin"] , return_counts=True)

plt.figure(figsize=(18,4))
plt.title("Electrical Component Events")
plt.xlabel("Time", fontdict=fontdict_text)
plt.ylabel("Number of Events", fontdict=fontdict_text)
plt.xticks(fontdict=fontdict_text)
sns.barplot(x=times, y=counts,color="b")
plt.show()

for component in np.unique(all_component_events.Component):
    component_events = all_component_events[all_component_events.Component == component].Timestamp.apply(lambda x: create_time_bin(x.hour, x.minute))
    times, counts = np.unique(component_events, return_counts=True)
    plt.figure(figsize=(18,4))
    plt.title(component + " Events")
    plt.xlabel("Time")
    plt.ylabel("Number of Events")
    sns.barplot(x=times, y=counts,color="b")
    plt.show()

## Electrical Component Events

## Mean instantenous power of the components

In [ ]:
mean_instant_power_list = []
component_list = []
x_axis = [] # x-axis, first component at 1, second at 2, third at 3
for index, component in enumerate( np.unique(all_component_events.Component), start=1):
    if component == "unlabeled": #skip the unlabeled ones
        continue
        
    component_events = all_component_events[all_component_events.Component == component]
    
    component_events = component_events.sample(n=100, random_state=10)
    # for efficienfy reasons, iterate over each day separately
    for day_date in np.unique(component_events.Date):
              
        for event in component_events[component_events.Date == day_date].itertuples():
            cream_day = CREAM_Day(cream_day_location=os.path.join(PATH_TO_DATA, str(day_date)), use_buffer=True, buffer_size_files=10)
            voltage, current = cream_day.load_time_frame(event.Timestamp, duration=0.3, return_noise=False)
            instant_power = voltage * current
            mean_instant_power_list.append(np.mean(instant_power))
            component_list.append(component)
            x_axis.append(index)
    


In [ ]:
import matplotlib
%matplotlib inline
component_list = np.array(component_list)
mean_instant_power_list = np.array(mean_instant_power_list)
fig, ax = plt.subplots(1, 3, figsize=(15,5), sharex=True, sharey=True)
#matplotlib.rcParams.update({'font.size': 10})
for i, component in enumerate(np.unique(component_list)):
    mask = np.where(component_list == component)[0]
    hist, bins = np.histogram(mean_instant_power_list[mask], bins=30)
    biggest_bin = np.argmax(hist) # get biggest bin and its value
    x, y, _ = ax[i].hist(mean_instant_power_list[mask], bins, color="b")
   
    max_bin = np.argmax(x)
    max_value = y[max_bin]
    
   # ax[i].set_xticklabels(y)
    
    ax[i].set_title(component, fontsize=18)
    ax[i].set_ylim(0,60)
    ax[i].set_ylabel("Samples", fontsize=16)
    ax[i].set_xlabel("Mean instantenous power,\n with maximum bin at value %.2f" %(max_value), fontsize=16)
    
fig.tight_layout()
plt.show()
fig.savefig("./component_mean_instant_power.pdf")